# **01. Import Libraries**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# **02. S&P 500**

In [ ]:
pip install yfinance

In [ ]:
# Define benchmark date range
start_date = '2016-01-01'
end_date = '2024-12-31'

# Download S&P 500 index with default multi-index columns
sp500 = yf.download('^GSPC', start=start_date, end=end_date, auto_adjust=False)

# Flatten MultiIndex columns if necessary
sp500.columns = [f"{col[0]}_{col[1]}" for col in sp500.columns]

#  Choose the correct close price column (using 'Adj Close' which is typically the adjusted close price)
price_col = 'Adj Close_^GSPC' if 'Adj Close_^GSPC' in sp500.columns else 'Close_^GSPC'

#  Calculate log returns
sp500['log_return'] = np.log(sp500[price_col] / sp500[price_col].shift(1))
sp500 = sp500.dropna()  # Drop any rows with NaN values

# Calculate cumulative log returns for S&P 500
sp500['cum_SP500_log_return'] = np.cumsum(sp500['log_return'])

# Ensure 'date' column is in datetime format
sp500['date'] = pd.to_datetime(sp500.index)

# Clean up and keep relevant columns
sp500 = sp500[[price_col, 'log_return', 'cum_SP500_log_return']].dropna()
sp500.reset_index(inplace=True)
sp500.rename(columns={'Date': 'date', price_col: 'close'}, inplace=True)

# Final result preview
print(sp500.head())


In [ ]:
sp500['log_return'] = np.log(sp500['close'] / sp500['close'].shift(1))
sp500 = sp500.dropna()

daily_returns = sp500['log_return'].values  # These are daily log returns

# Convert log returns to simple returns
simple_returns = np.expm1(daily_returns)  # exp(log_return) - 1 ≈ simple return

# Risk-free rate assumption
risk_free_rate = 0.01  # 1% annualized
periods = 252  # trading days

# Metric Functions
def cumulative_return(r):
    return np.prod(1 + r) - 1

def annualized_return(r):
    return (1 + cumulative_return(r)) ** (periods / len(r)) - 1

def sharpe_ratio(r):
    excess = r - risk_free_rate / periods
    return np.sqrt(periods) * np.mean(excess) / np.std(excess)

def calculate_volatility(r):
    return np.std(r) * np.sqrt(periods)

def maximum_drawdown(r):
    cum_returns = np.cumprod(1 + r)
    peak = np.maximum.accumulate(cum_returns)
    drawdown = (cum_returns - peak) / peak
    return np.min(drawdown)

# Compute Metrics
sp500_metrics = {
    'Cumulative Return': cumulative_return(simple_returns),
    'Annualized Return': annualized_return(simple_returns),
    'Sharpe Ratio': sharpe_ratio(simple_returns),
    'Volatility': calculate_volatility(simple_returns),
    'Standard Deviation': np.std(simple_returns),
    'Max Drawdown': maximum_drawdown(simple_returns)
}

# Display as DataFrame
sp500_metrics_df = pd.DataFrame(sp500_metrics, index=['S&P 500'])
display(sp500_metrics_df)


# **03. Window Size 5**

## Window Size 5 CR Plot (Without TC)

In [ ]:
# Define the file paths for each CSV
GLM_path = "/content/cumulative_log_returns_glm_lag_5.csv"
RF_path = "/content/cumulative_log_returns_rf_lag_5.csv"
GBRT_path = "/content/cumulative_log_returns_gbrt_lag_5.csv"
NN5_path = "/content/cumulative_log_returns_nn5_lag_5.csv"
Chronos_T5_Tiny_path = "/content/cumulative_log_chronost5tiny_lag_5.csv"
Chronos_Bolt_Tiny_path = "/content/cumulative_log_chronosbolttiny_lag_5.csv"
Chronos_Bolt_Mini_path = "/content/cumulative_log_chronosboltmini_lag_5.csv"
Chronos_Bolt_Small_path = "/content/cumulative_log_chronosboltsmall_lag_5.csv"
TimesFM1_path = "/content/cumulative_log_timesfm1_lag_5.csv"
TimesFM2_path = "/content/cumulative_log_timesfm2_lag_5.csv"
Moirai_Small_path = "/content/cumulative_log_returns_uni2tssmall_lag_5.csv"
Moirai_Small_Moe_path = "/content/cumulative_log_returns_uni2tssmallmoe_lag_5.csv"
Moirai_Base_Moe_path = "/content/cumulative_log_returns_uni2tsbasemoe_lag_5.csv"

# Read the CSV files into pandas DataFrames
GLM_data = pd.read_csv(GLM_path)
RF_data = pd.read_csv(RF_path)
GBRT_data = pd.read_csv(GBRT_path)
NN5_data = pd.read_csv(NN5_path)
Chronos_T5_Tiny_data = pd.read_csv(Chronos_T5_Tiny_path)
Chronos_Bolt_Tiny_data = pd.read_csv(Chronos_Bolt_Tiny_path)
Chronos_Bolt_Mini_data = pd.read_csv(Chronos_Bolt_Mini_path)
Chronos_Bolt_Small_data = pd.read_csv(Chronos_Bolt_Small_path)
TimesFM2_data = pd.read_csv(TimesFM2_path)
TimesFM1_data = pd.read_csv(TimesFM1_path)
Moirai_Small_data = pd.read_csv(Moirai_Small_path)
Moirai_Small_Moe_data = pd.read_csv(Moirai_Small_Moe_path)
Moirai_Base_Moe_data = pd.read_csv(Moirai_Base_Moe_path)

# Convert 'date' columns to datetime
GLM_data['date'] = pd.to_datetime(GLM_data['date'])
RF_data['date'] = pd.to_datetime(RF_data['date'])
GBRT_data['date'] = pd.to_datetime(GBRT_data['date'])
NN5_data['date'] = pd.to_datetime(NN5_data['date'])
Chronos_T5_Tiny_data['date'] = pd.to_datetime(Chronos_T5_Tiny_data['date'])
Chronos_Bolt_Tiny_data['date'] = pd.to_datetime(Chronos_Bolt_Tiny_data['date'])
Chronos_Bolt_Mini_data['date'] = pd.to_datetime(Chronos_Bolt_Mini_data['date'])
Chronos_Bolt_Small_data['date'] = pd.to_datetime(Chronos_Bolt_Small_data['date'])
TimesFM2_data['date'] = pd.to_datetime(TimesFM2_data['date'])
TimesFM1_data['date'] = pd.to_datetime(TimesFM1_data['date'])
Moirai_Small_data['date'] = pd.to_datetime(Moirai_Small_data['date'])
Moirai_Small_Moe_data['date'] = pd.to_datetime(Moirai_Small_Moe_data['date'])
Moirai_Base_Moe_data['date'] = pd.to_datetime(Moirai_Base_Moe_data['date'])

# Define the models dictionary with DataFrames
models = {
    "GLM": GLM_data,
    "RF": RF_data,
    "GBRT": GBRT_data,
    "NN5": NN5_data,
    "Chronos-T5 Tiny": Chronos_T5_Tiny_data,
    "Chronos-Bolt Tiny": Chronos_Bolt_Tiny_data,
    "Chronos-Bolt Mini": Chronos_Bolt_Mini_data,
    "Chronos-Bolt Small": Chronos_Bolt_Small_data,
    "TimesFM 2.0-500M": TimesFM2_data,
    "TimesFM 1.0-200M": TimesFM1_data,
    "Moirai Small": Moirai_Small_data,
    "Moirai-Moe Small": Moirai_Small_Moe_data,
    "Moirai-Moe Base": Moirai_Base_Moe_data
}

# Color palette
color_palette = sns.color_palette("tab20", n_colors=len(models))

def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_5_without_cost'] = model_data_filtered['cum_EL_return_5_without_cost']
        model_data_filtered.loc[:, 'cum_ES_return_5_without_cost'] = model_data_filtered['cum_ES_return_5_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_5_without_cost'],
                 color=color, linestyle='solid', label=f"{name} Long")
         # Plot the Short (Dashed)
         ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_5_without_cost'],
                 color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window Size 5 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag5_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_5_without_cost'] = model_data_filtered['cum_VL_return_5_without_cost']
        model_data_filtered.loc[:, 'cum_VS_return_5_without_cost'] = model_data_filtered['cum_VS_return_5_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_5_without_cost'],
                 color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_5_without_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 5 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag5_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window Size 5 CR Plot (With TC)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_5_with_cost'] = model_data_filtered['cum_EL_return_5_with_cost']
        model_data_filtered.loc[:, 'cum_ES_return_5_with_cost'] = model_data_filtered['cum_ES_return_5_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_5_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], -model_data_filtered['cum_ES_return_5_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window Size 5 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag5_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_5_with_cost'] = model_data_filtered['cum_VL_return_5_with_cost']
        model_data_filtered.loc[:, 'cum_VS_return_5_with_cost'] = model_data_filtered['cum_VS_return_5_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VL_return_5_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VS_return_5_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 5 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag5_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window Size 5 CR Plot (Equal-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_5_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 5 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag5_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_5_with_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 5 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)


    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag5_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window Size 5 CR Plot (Value-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_5_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Window Size 5 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag5_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Plot each model's combined long-short returns
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data.copy()

        # Plot the combined Long-Short returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_5_with_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Window Size 5 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag5_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

# **04. Window Size 21**

## Window 21 CR Plot (Without TC)

In [ ]:
# Define the file paths for each CSV
GLM_path = "/content/cumulative_log_returns_glm_lag_21.csv"
RF_path = "/content/cumulative_log_returns_rf_lag_21.csv"
GBRT_path = "/content/cumulative_log_returns_gbrt_lag_21.csv"
NN5_path = "/content/cumulative_log_returns_nn5_lag_21.csv"
Chronos_T5_Tiny_path = "/content/cumulative_log_chronost5tiny_lag_21.csv"
Chronos_Bolt_Tiny_path = "/content/cumulative_log_chronosbolttiny_lag_21.csv"
Chronos_Bolt_Mini_path = "/content/cumulative_log_chronosboltmini_lag_21.csv"
Chronos_Bolt_Small_path = "/content/cumulative_log_chronosboltsmall_lag_21.csv"
TimesFM1_path = "/content/cumulative_log_timesfm1_lag_21.csv"
TimesFM2_path = "/content/cumulative_log_timesfm2_lag_21.csv"
Moirai_Small_path = "/content/cumulative_log_returns_uni2tssmall_lag_21.csv"
Moirai_Small_Moe_path = "/content/cumulative_log_returns_uni2tssmallmoe_lag_21.csv"
Moirai_Base_Moe_path = "/content/cumulative_log_returns_uni2tsbasemoe_lag_21.csv"

# Read the CSV files into pandas DataFrames
GLM_data = pd.read_csv(GLM_path)
RF_data = pd.read_csv(RF_path)
GBRT_data = pd.read_csv(GBRT_path)
NN5_data = pd.read_csv(NN5_path)
Chronos_T5_Tiny_data = pd.read_csv(Chronos_T5_Tiny_path)
Chronos_Bolt_Tiny_data = pd.read_csv(Chronos_Bolt_Tiny_path)
Chronos_Bolt_Mini_data = pd.read_csv(Chronos_Bolt_Mini_path)
Chronos_Bolt_Small_data = pd.read_csv(Chronos_Bolt_Small_path)
TimesFM2_data = pd.read_csv(TimesFM2_path)
TimesFM1_data = pd.read_csv(TimesFM1_path)
Moirai_Small_data = pd.read_csv(Moirai_Small_path)
Moirai_Small_Moe_data = pd.read_csv(Moirai_Small_Moe_path)
Moirai_Base_Moe_data = pd.read_csv(Moirai_Base_Moe_path)

# Convert 'date' columns to datetime
GLM_data['date'] = pd.to_datetime(GLM_data['date'])
RF_data['date'] = pd.to_datetime(RF_data['date'])
GBRT_data['date'] = pd.to_datetime(GBRT_data['date'])
NN5_data['date'] = pd.to_datetime(NN5_data['date'])
Chronos_T5_Tiny_data['date'] = pd.to_datetime(Chronos_T5_Tiny_data['date'])
Chronos_Bolt_Tiny_data['date'] = pd.to_datetime(Chronos_Bolt_Tiny_data['date'])
Chronos_Bolt_Mini_data['date'] = pd.to_datetime(Chronos_Bolt_Mini_data['date'])
Chronos_Bolt_Small_data['date'] = pd.to_datetime(Chronos_Bolt_Small_data['date'])
TimesFM2_data['date'] = pd.to_datetime(TimesFM2_data['date'])
TimesFM1_data['date'] = pd.to_datetime(TimesFM1_data['date'])
Moirai_Small_data['date'] = pd.to_datetime(Moirai_Small_data['date'])
Moirai_Small_Moe_data['date'] = pd.to_datetime(Moirai_Small_Moe_data['date'])
Moirai_Base_Moe_data['date'] = pd.to_datetime(Moirai_Base_Moe_data['date'])

# Define the models dictionary with DataFrames
models = {
    "GLM": GLM_data,
    "RF": RF_data,
    "GBRT": GBRT_data,
    "NN5": NN5_data,
    "Chronos-T5 Tiny": Chronos_T5_Tiny_data,
    "Chronos-Bolt Tiny": Chronos_Bolt_Tiny_data,
    "Chronos-Bolt Mini": Chronos_Bolt_Mini_data,
    "Chronos-Bolt Small": Chronos_Bolt_Small_data,
    "TimesFM 2.0-500M": TimesFM2_data,
    "TimesFM 1.0-200M": TimesFM1_data,
    "Moirai Small": Moirai_Small_data,
    "Moirai-Moe Small": Moirai_Small_Moe_data,
    "Moirai-Moe Base": Moirai_Base_Moe_data
}

# Color palette
color_palette = sns.color_palette("tab20", n_colors=len(models))

def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_21_without_cost'] = model_data_filtered['cum_EL_return_21_without_cost']
        model_data_filtered.loc[:, 'cum_ES_return_21_without_cost'] = model_data_filtered['cum_ES_return_21_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_21_without_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_21_without_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window Size 21 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag21_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_21_without_cost'] = model_data_filtered['cum_VL_return_21_without_cost']
        model_data_filtered.loc[:, 'cum_VS_return_21_without_cost'] = model_data_filtered['cum_VS_return_21_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VL_return_21_without_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VS_return_21_without_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 21 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag21_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window Size 21 CR Plot (With TC)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_21_with_cost'] = model_data_filtered['cum_EL_return_21_with_cost']
        model_data_filtered.loc[:, 'cum_ES_return_21_with_cost'] = model_data_filtered['cum_ES_return_21_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_21_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_21_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window Size 21 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag21_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_21_with_cost'] = model_data_filtered['cum_VL_return_21_with_cost']
        model_data_filtered.loc[:, 'cum_VS_return_21_with_cost'] = model_data_filtered['cum_VS_return_21_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VL_return_21_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VS_return_21_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 21 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag21_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window 21 CR Plot (Equal-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_21_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 21 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)  # Add this legend inside the plot area

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag21_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_21_with_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 21 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)


    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag21_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window Size 21 CR Plot (Value-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_21_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Window Size 21 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag21_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Plot each model's combined long-short returns
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data.copy()

        # Plot the combined Long-Short returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_21_with_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Window Size 21 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag21_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

# **05. Window Size 252**

 ## Window size 252 CR Plot (Without TC)

In [ ]:
# Define the file paths for each CSV
GLM_path = "/content/cumulative_log_returns_glm_lag_252.csv"
RF_path = "/content/cumulative_log_returns_rf_lag_252.csv"
GBRT_path = "/content/cumulative_log_returns_gbrt_lag_252.csv"
NN5_path = "/content/cumulative_log_returns_nn5_lag_252.csv"
Chronos_T5_Tiny_path = "/content/cumulative_log_chronost5tiny_lag_252.csv"
Chronos_Bolt_Tiny_path = "/content/cumulative_log_chronosbolttiny_lag_252.csv"
Chronos_Bolt_Mini_path = "/content/cumulative_log_chronosboltmini_lag_252.csv"
Chronos_Bolt_Small_path = "/content/cumulative_log_chronosboltsmall_lag_252.csv"
TimesFM1_path = "/content/cumulative_log_timesfm1_lag_252.csv"
TimesFM2_path = "/content/cumulative_log_timesfm2_lag_252.csv"
Moirai_Small_path = "/content/cumulative_log_returns_uni2tssmall_lag_252.csv"
Moirai_Small_Moe_path = "/content/cumulative_log_returns_uni2tssmallmoe_lag_252.csv"
Moirai_Base_Moe_path = "/content/cumulative_log_returns_uni2tsbasemoe_lag_252.csv"

# Read the CSV files into pandas DataFrames
GLM_data = pd.read_csv(GLM_path)
RF_data = pd.read_csv(RF_path)
GBRT_data = pd.read_csv(GBRT_path)
NN5_data = pd.read_csv(NN5_path)
Chronos_T5_Tiny_data = pd.read_csv(Chronos_T5_Tiny_path)
Chronos_Bolt_Tiny_data = pd.read_csv(Chronos_Bolt_Tiny_path)
Chronos_Bolt_Mini_data = pd.read_csv(Chronos_Bolt_Mini_path)
Chronos_Bolt_Small_data = pd.read_csv(Chronos_Bolt_Small_path)
TimesFM2_data = pd.read_csv(TimesFM2_path)
TimesFM1_data = pd.read_csv(TimesFM1_path)
Moirai_Small_data = pd.read_csv(Moirai_Small_path)
Moirai_Small_Moe_data = pd.read_csv(Moirai_Small_Moe_path)
Moirai_Base_Moe_data = pd.read_csv(Moirai_Base_Moe_path)

# Convert 'date' columns to datetime
GLM_data['date'] = pd.to_datetime(GLM_data['date'])
RF_data['date'] = pd.to_datetime(RF_data['date'])
GBRT_data['date'] = pd.to_datetime(GBRT_data['date'])
NN5_data['date'] = pd.to_datetime(NN5_data['date'])
Chronos_T5_Tiny_data['date'] = pd.to_datetime(Chronos_T5_Tiny_data['date'])
Chronos_Bolt_Tiny_data['date'] = pd.to_datetime(Chronos_Bolt_Tiny_data['date'])
Chronos_Bolt_Mini_data['date'] = pd.to_datetime(Chronos_Bolt_Mini_data['date'])
Chronos_Bolt_Small_data['date'] = pd.to_datetime(Chronos_Bolt_Small_data['date'])
TimesFM2_data['date'] = pd.to_datetime(TimesFM2_data['date'])
TimesFM1_data['date'] = pd.to_datetime(TimesFM1_data['date'])
Moirai_Small_data['date'] = pd.to_datetime(Moirai_Small_data['date'])
Moirai_Small_Moe_data['date'] = pd.to_datetime(Moirai_Small_Moe_data['date'])
Moirai_Base_Moe_data['date'] = pd.to_datetime(Moirai_Base_Moe_data['date'])

# Define the models dictionary with DataFrames
models = {
    "GLM": GLM_data,
    "RF": RF_data,
    "GBRT": GBRT_data,
    "NN5": NN5_data,
    "Chronos-T5 Tiny": Chronos_T5_Tiny_data,
    "Chronos-Bolt Tiny": Chronos_Bolt_Tiny_data,
    "Chronos-Bolt Mini": Chronos_Bolt_Mini_data,
    "Chronos-Bolt Small": Chronos_Bolt_Small_data,
    "TimesFM 2.0-500M": TimesFM2_data,
    "TimesFM 1.0-200M": TimesFM1_data,
    "Moirai Small": Moirai_Small_data,
    "Moirai-Moe Small": Moirai_Small_Moe_data,
    "Moirai-Moe Base": Moirai_Base_Moe_data
}

# Color palette
color_palette = sns.color_palette("tab20", n_colors=len(models))

def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_252_without_cost'] = model_data_filtered['cum_EL_return_252_without_cost']
        model_data_filtered.loc[:, 'cum_ES_return_252_without_cost'] = model_data_filtered['cum_ES_return_252_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_252_without_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_252_without_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window Size 252 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag252_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range and smooth the data
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data by the specified date range
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_252_without_cost'] = model_data_filtered['cum_VL_return_252_without_cost']
        model_data_filtered.loc[:, 'cum_VS_return_252_without_cost'] = model_data_filtered['cum_VS_return_252_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VL_return_252_without_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VS_return_252_without_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 252 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag252_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window 252 CR Plot (With TC)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_252_with_cost'] = model_data_filtered['cum_EL_return_252_with_cost']
        model_data_filtered.loc[:, 'cum_ES_return_252_with_cost'] = model_data_filtered['cum_ES_return_252_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_252_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_252_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window Size 252 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag252_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_252_with_cost'] = model_data_filtered['cum_VL_return_252_with_cost']
        model_data_filtered.loc[:, 'cum_VS_return_252_with_cost'] = model_data_filtered['cum_VS_return_252_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VL_return_252_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VS_return_252_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 252 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag252_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window 252 CR Plot (Equal-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_252_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 252 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag252_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model for the desired date range
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_252_with_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 252 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)


    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag252_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window 252 CR Plot (Value-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_252_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Window Size 252 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag252_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Plot each model's combined long-short returns
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data.copy()

        # Plot the combined Long-Short returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_252_with_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Window Size 252 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag252_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

# **06. Window Size 512**

## Window 512 CR Plot (Without TC)

In [ ]:
# Define the file paths for each CSV
GLM_path = "/content/cumulative_log_returns_glm_lag_512.csv"
RF_path = "/content/cumulative_log_returns_rf_lag_512.csv"
GBRT_path = "/content/cumulative_log_returns_gbrt_lag_512.csv"
NN5_path = "/content/cumulative_log_returns_nn5_lag_512.csv"
Chronos_T5_Tiny_path = "/content/cumulative_log_chronost5tiny_lag_512.csv"
Chronos_Bolt_Tiny_path = "/content/cumulative_log_chronosbolttiny_lag_512.csv"
Chronos_Bolt_Mini_path = "/content/cumulative_log_chronosboltmini_lag_512.csv"
Chronos_Bolt_Small_path = "/content/cumulative_log_chronosboltsmall_lag_512.csv"
TimesFM1_path = "/content/cumulative_log_timesfm1_lag_512.csv"
TimesFM2_path = "/content/cumulative_log_timesfm2_lag_512.csv"
Moirai_Small_path = "/content/cumulative_log_returns_uni2tssmall_lag_512.csv"
Moirai_Small_Moe_path = "/content/cumulative_log_returns_uni2tssmallmoe_lag_512.csv"
Moirai_Base_Moe_path = "/content/cumulative_log_returns_uni2tsbasemoe_lag_512.csv"

# Read the CSV files into pandas DataFrames
GLM_data = pd.read_csv(GLM_path)
RF_data = pd.read_csv(RF_path)
GBRT_data = pd.read_csv(GBRT_path)
NN5_data = pd.read_csv(NN5_path)
Chronos_T5_Tiny_data = pd.read_csv(Chronos_T5_Tiny_path)
Chronos_Bolt_Tiny_data = pd.read_csv(Chronos_Bolt_Tiny_path)
Chronos_Bolt_Mini_data = pd.read_csv(Chronos_Bolt_Mini_path)
Chronos_Bolt_Small_data = pd.read_csv(Chronos_Bolt_Small_path)
TimesFM2_data = pd.read_csv(TimesFM2_path)
TimesFM1_data = pd.read_csv(TimesFM1_path)
Moirai_Small_data = pd.read_csv(Moirai_Small_path)
Moirai_Small_Moe_data = pd.read_csv(Moirai_Small_Moe_path)
Moirai_Base_Moe_data = pd.read_csv(Moirai_Base_Moe_path)

# Convert 'date' columns to datetime
GLM_data['date'] = pd.to_datetime(GLM_data['date'])
RF_data['date'] = pd.to_datetime(RF_data['date'])
GBRT_data['date'] = pd.to_datetime(GBRT_data['date'])
NN5_data['date'] = pd.to_datetime(NN5_data['date'])
Chronos_T5_Tiny_data['date'] = pd.to_datetime(Chronos_T5_Tiny_data['date'])
Chronos_Bolt_Tiny_data['date'] = pd.to_datetime(Chronos_Bolt_Tiny_data['date'])
Chronos_Bolt_Mini_data['date'] = pd.to_datetime(Chronos_Bolt_Mini_data['date'])
Chronos_Bolt_Small_data['date'] = pd.to_datetime(Chronos_Bolt_Small_data['date'])
TimesFM2_data['date'] = pd.to_datetime(TimesFM2_data['date'])
TimesFM1_data['date'] = pd.to_datetime(TimesFM1_data['date'])
Moirai_Small_data['date'] = pd.to_datetime(Moirai_Small_data['date'])
Moirai_Small_Moe_data['date'] = pd.to_datetime(Moirai_Small_Moe_data['date'])
Moirai_Base_Moe_data['date'] = pd.to_datetime(Moirai_Base_Moe_data['date'])

# Define the models dictionary with DataFrames
models = {
    "GLM": GLM_data,
    "RF": RF_data,
    "GBRT": GBRT_data,
    "NN5": NN5_data,
    "Chronos-T5 Tiny": Chronos_T5_Tiny_data,
    "Chronos-Bolt Tiny": Chronos_Bolt_Tiny_data,
    "Chronos-Bolt Mini": Chronos_Bolt_Mini_data,
    "Chronos-Bolt Small": Chronos_Bolt_Small_data,
    "TimesFM 2.0-500M": TimesFM2_data,
    "TimesFM 1.0-200M": TimesFM1_data,
    "Moirai Small": Moirai_Small_data,
    "Moirai-Moe Small": Moirai_Small_Moe_data,
    "Moirai-Moe Base": Moirai_Base_Moe_data
}

# Color palette
color_palette = sns.color_palette("tab20", n_colors=len(models))

def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_512_without_cost'] = model_data_filtered['cum_EL_return_512_without_cost']
        model_data_filtered.loc[:, 'cum_ES_return_512_without_cost'] = model_data_filtered['cum_ES_return_512_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_512_without_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_512_without_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window Size 512 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag512_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_512_without_cost'] = model_data_filtered['cum_VL_return_512_without_cost']
        model_data_filtered.loc[:, 'cum_VS_return_512_without_cost'] = model_data_filtered['cum_VS_return_512_without_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VL_return_512_without_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VS_return_512_without_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 512 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag512_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window 512 CR Plot (With TC)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_EL_return_512_with_cost'] = model_data_filtered['cum_EL_return_512_with_cost']
        model_data_filtered.loc[:, 'cum_ES_return_512_with_cost'] = model_data_filtered['cum_ES_return_512_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_EL_return_512_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ES_return_512_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Window 512 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Equal-Weighted_Portfolio_Performance_Lag512_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        model_data_filtered.loc[:, 'cum_VL_return_512_with_cost'] = model_data_filtered['cum_VL_return_512_with_cost']
        model_data_filtered.loc[:, 'cum_VS_return_512_with_cost'] = model_data_filtered['cum_VS_return_512_with_cost']

        # Plot the Long (Solid)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VL_return_512_with_cost'],
                color=color, linestyle='solid', label=f"{name} Long")
        # Plot the Short (Dashed)
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VS_return_512_with_cost'],
                color=color, linestyle='dashed', label=f"{name} Short")

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Window Size 512 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for model colors
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Legend for line styles
    style_lines = [
        Line2D([0], [0], color='black', linestyle='solid', linewidth=2),
        Line2D([0], [0], color='black', linestyle='dashed', linewidth=2)
    ]
    ax.legend(style_lines, ['Long', 'Short'], loc='upper right', fontsize=9,
              frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Value-Weighted_Portfolio_Performance_Lag512_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Wndow 512 CR Plot (Equal-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_512_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 512 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag512_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_ELS_return_512_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Equal-Weighted Portfolio Performance (Long-Short) (Window Size 512 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag512_Cost.png"
plot_portfolio_performance(models, sp500, save_path)

## Window 512 CR Plot (Value-Weighted) (LS)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None, start_date='2016-01-05', end_date='2024-12-31'):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_512_without_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Lag 512 - No Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage with the models
save_path = "Long-Short_Portfolio_Performance_Lag512_NoCost.png"
plot_portfolio_performance(models, sp500, save_path)

In [ ]:
def plot_portfolio_performance(models, sp500_data, save_path=None):
    fig, ax = plt.subplots(figsize=(20, 10))

    # Filter each model
    for i, (name, model_data) in enumerate(models.items()):
        color = color_palette[i]

        # Filter the model data
        model_data_filtered = model_data[(model_data['date'] >= start_date) & (model_data['date'] <= end_date)].copy()

        # Plot the returns
        ax.plot(model_data_filtered['date'], model_data_filtered['cum_VLS_return_512_with_cost'],
                color=color, linestyle='solid', label=f'{name} (Long-Short)')

    # S&P 500 benchmark
    sp500_line, = ax.plot(sp500_data['date'], sp500_data['cum_SP500_log_return'],
                          color='black', linewidth=2, label='S&P 500')

    # Horizontal zero line
    ax.axhline(0, color='black', linestyle=':', linewidth=1)

    # Y-axis: numeric values
    ax.set_ylabel("Cumulative Log Return", fontsize=12)
    ax.tick_params(axis='y', labelsize=10)

    # X-axis label
    ax.set_xlabel("Date", fontsize=12)

    # Title
    ax.set_title("Value-Weighted Portfolio Performance (Long-Short) (Lag 512 - Transaction Cost)", fontsize=14)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.5)

    # Legend for models and S&P 500
    model_handles = [Line2D([0], [0], color=color_palette[i], linewidth=3) for i in range(len(models))]
    model_labels = list(models.keys())

    # Add the S&P 500 to the model legend
    model_handles.append(sp500_line)
    model_labels.append('S&P 500')

    # Model legend in the upper left
    legend1 = ax.legend(model_handles, model_labels, loc='upper left', fontsize=10,
                        frameon=True, fancybox=True, facecolor='white', edgecolor='black', framealpha=0.8)
    ax.add_artist(legend1)

    # Layout adjustment for bottom space
    plt.tight_layout(pad=5.0, rect=[0, 0, 1, 0.9])

    # Save or show
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

# Example usage
save_path = "Long-Short_Portfolio_Performance_Lag512_Cost.png"
plot_portfolio_performance(models, sp500, save_path)